In [11]:
from pyspark import SparkContext
sc=SparkContext()

In [4]:
import re
import datetime
import sys
import os


In [39]:
data = sc.parallelize([("userid1","This is a tweet","tweets", "timestamp1", "source1"),("userid2","content2","content_type2", "timestamp2", "source2")])

In [134]:
rdd=sc.textFile("dolce_processed.txt")
rdd.collect()

[u'"blazar30","11787678","","bio","2017-12-02T16:23:24","twitter"',
 u'"anman30031","11787678","","bio","2017-12-02T16:23:37","twitter"',
 u'"Dolce_Innocent2","11787678","","bio","2017-12-02T16:23:53","twitter"',
 u'"AriAngioloni","11787678","img classEmoji EmojiforText srchttpsabstwimgcomemojiv272x722b50png draggablefalse alt\u2b50\ufe0f titleMedium star arialabelEmoji Medium starJIKOOK IS REAL","bio","2017-12-02T16:24:05","twitter"',
 u'"SmokeeRobinson","11787678","Also known as Joe the Policeman in the quotWhat39s Goin Downquot episode of quotThat39s My Mamaquot PSN  MrWarden1117","bio","2017-12-02T16:24:07","twitter"',
 u'"StayFoodit","11787678","Amiamo l39Agroalimentare e il made in Italy abbiamo creato un centro commerciale online di prodotti tipici di nicchia a hrefstayfood classtweeturl twitteratreply prettylink dirltr datamentioneduserid0 relnofollow ssbstayfoodbait  a hrefhashtagfoodsrchash dataquerysourcehashtagclick classtwitterhashtag prettylink jsnav dirltr ssbfoodba a hr

In [157]:
##TWITTER_PATH = sc.filename('miumiu_processed.txt')
DATAFILE_PATTERN = re.compile(r'^"(.*?)","(.*?)","(.*?)","(.*?)","(.*?)","(.*?)"$')
from pyspark.sql import Row
import datetime

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[0:4]),
                             int(s[5:7]),
                             int(s[8:10]),
                             int(s[11:13]),
                             int(s[14:16]),
                             int(s[17:19]))

def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    #match = re.search(DATAFILE_PATTERN, logline)
    match = [s.replace("\"", "") for s in logline.split(",")]
    
    return match
    '''
    return (Row(
        UID          = match[0],
        Item_ID  = match[1],
        content       = match[2],
        content_type = match[3],
        date_time     = parse_apache_time(match[4]),
        source        = match[5]
    ), 1)
'''
DataRDD = (sc.textFile('miumiu_processed.txt').map(parseApacheLogLine).cache())


TweetsRDD = DataRDD.filter(lambda (a,b,c,d,e,f): d=='tweet')
RetweetCountsRDD = DataRDD.filter(lambda (a,b,c,d,e,f): d=='retweet_counts')
ReplyCountsRDD = DataRDD.filter(lambda (a,b,c,d,e,f): d=='reply_counts')
LikeCountsRDD = DataRDD.filter(lambda (a,b,c,d,e,f): d=='like_counts')

In [158]:

TweetsRDD.take(3)

[[u'tiffany_thornt',
  u'12143286',
  u'MIU MIU Shearling Fur Skin Hooded Coat  Leather Trim New wo Tags CamelSz 40  Full read by eBay Price 5130\u2026 httpebayto1BYRKpn ',
  u'tweet',
  u'2014-11-22T20:45:10',
  u'twitter'],
 [u'OnMySpaceship',
  u'12143286',
  u'httpifttt1YSBX2z  NWOT Stunning  classic Miu Miu cotton and mink fur trim coat size 40  perfecto',
  u'tweet',
  u'2016-04-26T22:33:12',
  u'twitter'],
 [u'OnMySpaceship',
  u'12143286',
  u'httpifttt2byZsB6  NWOT Stunning  classic Miu Miu cotton and mink fur trim coat size 40  perfecto',
  u'tweet',
  u'2016-08-30T21:45:55',
  u'twitter']]

In [159]:
def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')


In [160]:
split_regex = r'[a-zA-Z0-9_]+'

def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    string = re.sub(r'([^\s\w(?<_)])+', ' ',string.lower())
    return re.findall(split_regex, string)

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    
    def remove_if_not_substring(l1, l2):
        return [i for i in l1 if not any(j == i for j in list(l2))]
    string = remove_if_not_substring(simpleTokenize(string), stopwords)
    return string

In [161]:
baseDir = os.path.join('data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())

In [162]:
TweetsTokenizedRDD = TweetsRDD.map(lambda (a,b,c,d,e,f): (a,b,tokenize(c),d,e,f))


In [163]:
def tf(tokens):
    """ Compute TF
    Args:
        tokens (list of str): input list of tokens from tokenize
    Returns:
        dictionary: a dictionary of tokens to its TF values
    """
    total_len = float(len(tokens))
    dictionary = {}
    for x in tokens:
        if x in dictionary.keys():
            dictionary[x] = dictionary[x]+1/total_len
        else:
            dictionary[x] = 1/total_len
    return dictionary ##dict(zip(tokens,[float(len(x))/total_len for x in tokens]))

In [164]:
def idfs(corpus):
    """ Compute IDF
    Args:
        corpus (RDD): input corpus
    Returns:
        RDD: a RDD of (token, IDF value)
        """
    N = corpus.count()
    uniqueTokens = corpus.map(lambda ut: list(set(ut[1])))
    tokenCountPairTuple = uniqueTokens.flatMap(lambda strg: [(key, 1) for key in strg]).reduceByKey(lambda x, y: x + y)
    tokenSumPairTuple = tokenCountPairTuple.map(lambda duple: (duple[0],   float(N) / duple[1]))
    return (tokenSumPairTuple)

In [165]:
TweetsRecTok = TweetsTokenizedRDD.map(lambda (a,b,c,d,e,f):(a,c))
print TweetsRecTok.take(100)
##TweetsRecTok.map(lambda x: set)
idf = idfs(TweetsRecTok)
print idf.collect()

[(u'tiffany_thornt', [u'miu', u'miu', u'shearling', u'fur', u'skin', u'hooded', u'coat', u'leather', u'trim', u'new', u'wo', u'tags', u'camelsz', u'40', u'full', u'read', u'ebay', u'price', u'5130', u'httpebayto1byrkpn']), (u'OnMySpaceship', [u'httpifttt1ysbx2z', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton', u'mink', u'fur', u'trim', u'coat', u'size', u'40', u'perfecto']), (u'OnMySpaceship', [u'httpifttt2byzsb6', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton', u'mink', u'fur', u'trim', u'coat', u'size', u'40', u'perfecto']), (u'OnMySpaceship', [u'httpifttt2dejmst', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton', u'mink', u'fur', u'trim', u'coat', u'size', u'40', u'perfecto']), (u'OnMySpaceship', [u'httpifttt2dog4vb', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton', u'mink', u'fur', u'trim', u'coat', u'size', u'40', u'perfecto'])]
[(u'trim', 1.0), (u'mink', 1.25), (u'classic', 1.25), (u'skin', 5.0), (u'httpebayto1byrkpn', 5.0), (u'htt

In [166]:
idf.count()

30

In [167]:
def tfidf(tokens, idfs):
    """ Compute TF-IDF
    Args:
        tokens (list of str): input list of tokens from tokenize
        idfs (dictionary): record to IDF value
    Returns:
        dictionary: a dictionary of records to TF-IDF values
    """
    tfs = tf(tokens)
    tfIdfDict = {}
    for i in tokens:
        tfIdfDict[i]=tfs[i]*idfs[i]
    return tfIdfDict



In [168]:
import math

def dotprod(a, b):
    
    return sum([a[i]*b[i] for i in a.keys() if any(j == i for j in b.keys())])

def norm(a):
    
    return math.sqrt(dotprod(a,a))

def cossim(a, b):

    return dotprod(a,b)/(norm(a)*norm(b))

In [169]:
def cosineSimilarity(string1, string2, idfsDictionary):

    w1 = tfidf(tokenize(string1), idfsDictionary)
    w2 = tfidf(tokenize(string2), idfsDictionary)
    return cossim(w1, w2)

In [170]:
def computeSimilarity(record):
    """ Compute similarity on a combination record
    Args:
        record: a pair, (google record, amazon record)
    Returns:
        pair: a pair, (google URL, amazon ID, cosine similarity value)
    """
    googleRec = record[0]
    amazonRec = record[1]
    googleURL = googleRec[0]
    amazonID = amazonRec[0]
    googleValue = googleRec[1]
    amazonValue = amazonRec[1]
    cs = cosineSimilarity(googleValue,amazonValue, idfsSmallWeights)
    return (googleURL, amazonID, cs)



def similar(amazonID, googleURL):
    """ Return similarity value
    Args:
        amazonID: amazon ID
        googleURL: google URL
    Returns:
        similar: cosine similarity value
    """
    return (similarities
            .filter(lambda record: (record[0] == googleURL and record[1] == amazonID))
            .collect()[0][2])

from pyspark import Broadcast


In [171]:
def computeSimilarityBroadcast(record):
    """ Compute similarity on a combination record, using Broadcast variable
    Args:
        record: a pair, (google record, amazon record)
    Returns:
        pair: a pair, (google URL, amazon ID, cosine similarity value)
    """
    googleRec = record[0]
    amazonRec = record[1]
    googleURL = googleRec[0]
    amazonID = amazonRec[0]
    googleValue = googleRec[1]
    amazonValue = amazonRec[1]
    cs = cosineSimilarity(googleValue,amazonValue, idfsSmallBroadcast.value)
    return (googleURL, amazonID, cs)

def similarBroadcast(amazonID, googleURL):
    """ Return similarity value, computed using Broadcast variable
    Args:
        amazonID: amazon ID
        googleURL: google URL
    Returns:
        similar: cosine similarity value
    """
    return (similaritiesBroadcast
            .filter(lambda record: (record[0] == googleURL and record[1] == amazonID))
            .collect()[0][2])

In [172]:
fullCorpusRDD = TweetsRecTok
idfsFull = idfs(fullCorpusRDD)
idfsFullWeights = idfsFull.collectAsMap()
print idfsFullWeights
idfsFullBroadcast = sc.broadcast(idfsFullWeights)
print TweetsRecTok.take(3)
TweetWeightRDD = TweetsRecTok.map(lambda (a,b):(a,tfidf(b, idfsFullWeights)))

{u'trim': 1.0, u'mink': 1.25, u'classic': 1.25, u'skin': 5.0, u'httpebayto1byrkpn': 5.0, u'httpifttt2byzsb6': 5.0, u'cotton': 1.25, u'hooded': 5.0, u'size': 1.25, u'40': 1.0, u'read': 5.0, u'httpifttt1ysbx2z': 5.0, u'nwot': 1.25, u'new': 5.0, u'5130': 5.0, u'miu': 1.0, u'full': 5.0, u'tags': 5.0, u'wo': 5.0, u'price': 5.0, u'shearling': 5.0, u'httpifttt2dejmst': 5.0, u'camelsz': 5.0, u'fur': 1.0, u'leather': 5.0, u'coat': 1.0, u'perfecto': 1.25, u'ebay': 5.0, u'stunning': 1.25, u'httpifttt2dog4vb': 5.0}
[(u'tiffany_thornt', [u'miu', u'miu', u'shearling', u'fur', u'skin', u'hooded', u'coat', u'leather', u'trim', u'new', u'wo', u'tags', u'camelsz', u'40', u'full', u'read', u'ebay', u'price', u'5130', u'httpebayto1byrkpn']), (u'OnMySpaceship', [u'httpifttt1ysbx2z', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton', u'mink', u'fur', u'trim', u'coat', u'size', u'40', u'perfecto']), (u'OnMySpaceship', [u'httpifttt2byzsb6', u'nwot', u'stunning', u'classic', u'miu', u'miu', u'cotton'

In [173]:
TweetWeightRDD.take(3)

[(u'tiffany_thornt',
  {u'40': 0.05,
   u'5130': 0.25,
   u'camelsz': 0.25,
   u'coat': 0.05,
   u'ebay': 0.25,
   u'full': 0.25,
   u'fur': 0.05,
   u'hooded': 0.25,
   u'httpebayto1byrkpn': 0.25,
   u'leather': 0.25,
   u'miu': 0.1,
   u'new': 0.25,
   u'price': 0.25,
   u'read': 0.25,
   u'shearling': 0.25,
   u'skin': 0.25,
   u'tags': 0.25,
   u'trim': 0.05,
   u'wo': 0.25}),
 (u'OnMySpaceship',
  {u'40': 0.07142857142857142,
   u'classic': 0.08928571428571427,
   u'coat': 0.07142857142857142,
   u'cotton': 0.08928571428571427,
   u'fur': 0.07142857142857142,
   u'httpifttt1ysbx2z': 0.3571428571428571,
   u'mink': 0.08928571428571427,
   u'miu': 0.14285714285714285,
   u'nwot': 0.08928571428571427,
   u'perfecto': 0.08928571428571427,
   u'size': 0.08928571428571427,
   u'stunning': 0.08928571428571427,
   u'trim': 0.07142857142857142}),
 (u'OnMySpaceship',
  {u'40': 0.07142857142857142,
   u'classic': 0.08928571428571427,
   u'coat': 0.07142857142857142,
   u'cotton': 0.089285714